In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = keras.Sequential([
    layers.Dense(16, activation="relu", input_shape=(4,)),
    layers.Dense(8, activation="relu"),
    layers.Dense(3, activation="softmax")
])

In [ ]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
class CustomModelCheckpoint(keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_acc = 0.0  # track best validation accuracy

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_acc = logs.get("val_accuracy")

        if val_acc is not None and val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            # Convert val_acc to percentage format like 95
            acc_percent = int(val_acc * 100)
            filename = f"dnn_model_val_acc_{acc_percent}.keras"
            self.model.save(filename)
            print(f"\nModel improved to {acc_percent}% accuracy. Saved as {filename}.")

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=8,
    callbacks=[CustomModelCheckpoint()],
    verbose=1
)